In [1]:
import pandas as pd
import sqlite3 as sql
from sqlalchemy import MetaData, Table, Column, Integer, String, Float, DateTime, ForeignKey, create_engine, insert

In [3]:
conn = sql.connect('../../../../Analytics-Jumpstart_10-5-19/data/weather.db')
weather_2013_df = pd.read_sql_query("SELECT * FROM weather2013;", conn)

In [5]:
weather_2013_df = weather_2013_df.loc[1:, :]

In [40]:
weather_2013_df.shape

(36246, 8)

In [15]:
weather_2013_station_data = weather_2013_df[['station', 'name', 'lat', 'lng', 'elevation']].drop_duplicates().reset_index(drop=True)
weather_2013_rain_data = weather_2013_df[['station', 'date', 'precip', 'avg_temp']].reset_index()

In [20]:
weather_2013_station_data

,station,name,lat,lng,elevation
0,US1TNCH0002,"KINGSTON SPRINGS 0.3 ENE, TN US",36.0984,-87.0963,177.1
1,US1TNCH0001,"ASHLAND CITY 5.6 ENE, TN US",36.306,-86.966,232.9
2,USC00403280,"FRANKLIN SEWAGE PLANT, TN US",35.9417,-86.8685,199.6
3,US1TNCH0007,"KINGSTON SPRINGS 1.4 SW, TN US",36.0806,-87.1165,224
4,US1TNCH0004,"KINGSTON SPRINGS 4.8 SW, TN US",36.0556,-87.1701,238
...,...,...,...,...,...
150,US1TNDV0122,"NASHVILLE 4.8 NE, TN US",36.213,-86.7166,138.7
151,US1TNDV0083,"BRENTWOOD 4.2 ENE, TN US",36.0282,-86.7228,253.9
152,US1TNDV0086,"GOODLETTSVILLE 1.1 SW, TN US",36.3221,-86.7173,154.8
153,US1TNDV0082,"OLD HICKORY 1.2 SSE, TN US",36.2436,-86.6396,146


In [24]:
weather_2013_rain_data.columns = ['record_id', 'station', 'date', 'precip', 'avg_temp']

In [25]:
weather_2013_rain_data.head()

,record_id,station,date,precip,avg_temp
0,1,US1TNCH0002,1/1/13,0.38,
1,2,US1TNCH0002,1/2/13,0.08,
2,3,US1TNCH0002,1/3/13,0,
3,4,US1TNCH0002,1/4/13,0,
4,5,US1TNCH0002,1/5/13,0,


In [8]:
def make_table_input(df, table_name, primary_key = None, foreign_key = None):
    '''
    Function to print out the function input to make a sqlalchemy table
    
    foreign_key arg requires a list of tuples or lists with 2 elements: (col_name, reference_df.reference_col_name)
    '''
    column_list = []
    for c in df.columns:
        arg = "Column('{}', {}".format(c, df[c].dtype)
        if foreign_key is not None:
            for f_key in foreign_key:
                if f_key[0]==c:
                    arg += ", ForeignKey('{}')".format(f_key[1])
        if primary_key == c:
            arg += ", primary_key=True"
        arg += ")"
        column_list.append(arg)
    concat_args = ", \n".join(column_list).replace('object', 'String').replace('float64', 'Float').replace('int64', 'Integer').replace('datetime64[ns]', 'DateTime')
    return "Table('{}', metadata, \n".format(table_name)+concat_args+')'

In [27]:
print(make_table_input(weather_2013_station_data, 'weather_stations', primary_key = 'station'))
print('\n')
print(make_table_input(weather_2013_rain_data, 'rainfall_2013', primary_key = 'record_id', foreign_key = [('station', 'weather_stations.station')]))


Table('weather_stations', metadata, 
Column('station', String, primary_key=True), 
Column('name', String), 
Column('lat', String), 
Column('lng', String), 
Column('elevation', String))


Table('rainfall_2013', metadata, 
Column('record_id', Integer, primary_key=True), 
Column('station', String, ForeignKey('weather_stations.station')), 
Column('date', String), 
Column('precip', String), 
Column('avg_temp', String))


In [28]:
db_uri = 'sqlite:///weather.db'
engine = create_engine(db_uri)
connection = engine.connect()
metadata = MetaData(engine)

weather_stations_table = Table('weather_stations', metadata, 
Column('station', String, primary_key=True), 
Column('name', String), 
Column('lat', String), 
Column('lng', String), 
Column('elevation', String))

rainfall_2013_table = Table('rainfall_2013', metadata, 
Column('record_id', Integer, primary_key=True), 
Column('station', String, ForeignKey('weather_stations.station')), 
Column('date', String), 
Column('precip', String), 
Column('avg_temp', String))

metadata.create_all(engine)

for table, df in zip([weather_stations_table, rainfall_2013_table], [weather_2013_station_data, weather_2013_rain_data]):
    print(table)
    query = insert(table)
    values_list = list(df.to_dict(orient='index').values())
    ResultProxy = connection.execute(query,values_list)

weather_stations
rainfall_2013


In [29]:
conn = sql.connect('weather.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
available_tables = (cur.fetchall())
print(available_tables)

[('rainfall_2013',), ('weather_stations',)]


In [33]:
pd.read_sql_query("SELECT * FROM rainfall_2013 LIMIT 5;", conn)

,record_id,station,date,precip,avg_temp
0,1,US1TNCH0002,1/1/13,0.38,
1,2,US1TNCH0002,1/2/13,0.08,
2,3,US1TNCH0002,1/3/13,0.0,
3,4,US1TNCH0002,1/4/13,0.0,
4,5,US1TNCH0002,1/5/13,0.0,


In [41]:
query = '''
SELECT weather_stations.*, rainfall_2013.*
FROM rainfall_2013
LEFT JOIN weather_stations ON weather_stations.station = rainfall_2013.station
ORDER BY rainfall_2013.date
'''

In [42]:
pd.read_sql(query, conn)

,station,name,lat,lng,elevation,record_id,station,date,precip,avg_temp
0,US1TNCH0002,"KINGSTON SPRINGS 0.3 ENE, TN US",36.0984,-87.0963,177.1,1,US1TNCH0002,1/1/13,0.38,
1,US1TNCH0001,"ASHLAND CITY 5.6 ENE, TN US",36.306,-86.966,232.9,331,US1TNCH0001,1/1/13,0.37,
2,USC00403280,"FRANKLIN SEWAGE PLANT, TN US",35.9417,-86.8685,199.6,673,USC00403280,1/1/13,0.41,
3,US1TNCH0007,"KINGSTON SPRINGS 1.4 SW, TN US",36.0806,-87.1165,224.0,1036,US1TNCH0007,1/1/13,0.39,
4,US1TNCH0004,"KINGSTON SPRINGS 4.8 SW, TN US",36.0556,-87.1701,238.0,1357,US1TNCH0004,1/1/13,0.39,
...,...,...,...,...,...,...,...,...,...,...
36241,US1TNWN0014,"GREEN HILL 1.7 S, TN US",36.2104,-86.5736,164.0,34629,US1TNWN0014,9/9/13,0.03,
36242,US1TNDV0121,"NASHVILLE 6.4 E, TN US",36.1784,-86.6704,188.4,35250,US1TNDV0121,9/9/13,0.0,
36243,US1TNDV0087,"BERRY HILL 2.3 W, TN US",36.1169,-86.8098,189.9,35389,US1TNDV0087,9/9/13,0.07,
36244,US1TNDV0120,"HERMITAGE 0.9 WSW, TN US",36.1903,-86.6378,143.9,35662,US1TNDV0120,9/9/13,0.01,
